<a href="https://colab.research.google.com/github/himanshu63867/Machine-Learning-Projects/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
#printing the stop wordsin English
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [32]:
#loading the dataset to a padas dataframe
df=pd.read_csv("/content/news.csv")

In [33]:
#check number of rows and columns
df.shape


(6335, 4)

In [34]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [35]:
df['label'] = df['label'].replace('REAL', '0')
df['label'] = df['label'].replace('FAKE', '1')
df['label'] = df['label'].astype('int32')

In [36]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0


In [37]:
df.isnull().sum()


,0
Unnamed: 0,0
title,0
text,0
label,0


In [38]:
#removing column Unnamed
df.drop(columns=["Unnamed: 0"],axis=1,inplace=True)

In [25]:
df.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0


In [39]:
#separating the data and label
X=df.drop(columns="label",axis=1)
Y=df["label"]

In [40]:
print(X)
print(Y)

                                                  title  \
0                          You Can Smell Hillary’s Fear   
1     Watch The Exact Moment Paul Ryan Committed Pol...   
2           Kerry to go to Paris in gesture of sympathy   
3     Bernie supporters on Twitter erupt in anger ag...   
4      The Battle of New York: Why This Primary Matters   
...                                                 ...   
6330  State Department says it can't find emails fro...   
6331  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332  Anti-Trump Protesters Are Tools of the Oligarc...   
6333  In Ethiopia, Obama seeks progress on peace, se...   
6334  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text  
0     Daniel Greenfield, a Shillman Journalism Fello...  
1     Google Pinterest Digg Linkedin Reddit Stumbleu...  
2     U.S. Secretary of State John F. Kerry said Mon...  
3     — Kaydee King (@KaydeeKing) November 9, 2016 T...  
4

Stemming:
is the procees of reducing a word to its root word
example:actor,actress,acting-->act

In [53]:
port_stem=PorterStemmer()


In [54]:
def stemming(title):
  stemmed_title=re.sub('[^a-zA-Z]','',title)
  stemmed_title=stemmed_title.lower()
  stemmed_title=stemmed_title.split()
  stemmed_title=[port_stem.stem(word) for word in stemmed_title if not word in stopwords.words("english")]
  stemmed_title=''.join(stemmed_title)
  return stemmed_title

In [55]:
df['title']=df['title'].apply(stemming)

In [56]:
print(df['title'])

0                                 youcansmellhillarysfear
1       watchtheexactmomentpaulryancommittedpoliticals...
2                     kerrytogotoparisingestureofsympathi
3       berniesupportersontwittereruptinangeragainstth...
4                    thebattleofnewyorkwhythisprimarymatt
                              ...                        
6330    statedepartmentsaysitcantfindemailsfromclinton...
6331         thepinpbsshouldstandforplutocraticorpentagon
6332      antitrumpprotestersaretoolsoftheoligarchyinform
6333    inethiopiaobamaseeksprogressonpeacesecurityine...
6334      jebbushissuddenlyattackingtrumphereswhythatmatt
Name: title, Length: 6335, dtype: object


In [57]:
#separating the data and label
X=df['title'].values
Y=df['label'].values

In [58]:
print(X)
print(Y)

['youcansmellhillarysfear'
 'watchtheexactmomentpaulryancommittedpoliticalsuicideatatrumprallyvideo'
 'kerrytogotoparisingestureofsympathi' ...
 'antitrumpprotestersaretoolsoftheoligarchyinform'
 'inethiopiaobamaseeksprogressonpeacesecurityineastafrica'
 'jebbushissuddenlyattackingtrumphereswhythatmatt']
[1 1 0 ... 1 0 0]


In [59]:
Y.shape

(6335,)

In [60]:
#converting textual adata to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(X)

X=vectorizer.transform(X)

In [61]:
print(X)

  (0, 6192)	1.0
  (1, 5748)	1.0
  (2, 2842)	1.0
  (3, 430)	1.0
  (4, 4742)	1.0
  (5, 4696)	1.0
  (6, 1869)	1.0
  (7, 593)	1.0
  (8, 1616)	1.0
  (9, 2545)	1.0
  (10, 6126)	1.0
  (11, 1394)	1.0
  (12, 4576)	1.0
  (13, 5750)	1.0
  (14, 5434)	1.0
  (15, 2380)	1.0
  (16, 4433)	1.0
  (17, 2095)	1.0
  (18, 5848)	1.0
  (19, 4756)	1.0
  (20, 5035)	1.0
  (21, 1526)	1.0
  (22, 2104)	1.0
  (23, 3346)	1.0
  (24, 2464)	1.0
  :	:
  (6310, 2597)	1.0
  (6311, 4753)	1.0
  (6312, 24)	1.0
  (6313, 1332)	1.0
  (6314, 4068)	1.0
  (6315, 3782)	1.0
  (6316, 1380)	1.0
  (6317, 1696)	1.0
  (6318, 542)	1.0
  (6319, 6026)	1.0
  (6320, 3410)	1.0
  (6321, 2957)	1.0
  (6322, 3969)	1.0
  (6323, 420)	1.0
  (6324, 77)	1.0
  (6325, 6112)	1.0
  (6326, 1311)	1.0
  (6327, 1939)	1.0
  (6328, 3991)	1.0
  (6329, 283)	1.0
  (6330, 4535)	1.0
  (6331, 4961)	1.0
  (6332, 202)	1.0
  (6333, 2463)	1.0
  (6334, 2718)	1.0


Splitting the dataset to Training and test data

In [63]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)


Training the model: Logistic Regression

In [65]:
model=LogisticRegression()

In [66]:
model.fit(X_train,Y_train)

LogisticRegression()

Evaluation  : accuracy score

In [67]:
#accuracy score onthe training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [68]:
print("Accuracy score of the training data:",training_data_accuracy)

Accuracy score of the training data: 0.999605367008682


In [69]:
#accuracy score onthe test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [70]:
print("Accuracy score of the test data:",test_data_accuracy)

Accuracy score of the test data: 0.5177584846093133


Make a Predictive System

In [81]:
X_news=X_test[2]
prediction=model.predict(X_news)
print(prediction)

if (prediction[0]==0):
  print("The news is Real")
else:
  print("The news is Fake")


[0]
The news is Real


In [82]:
print(Y_test[2])

0
